In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"praveengadiyaram","key":"22e59527e71b719aec4041c3be01c7ec"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-05-21 06:23:33           5930  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           3304  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1167  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           1914  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [6]:
! kaggle datasets download -d danielwillgeorge/glove6b100dtxt

 94% 123M/131M [00:01<00:00, 103MB/s] 
100% 131M/131M [00:01<00:00, 109MB/s]


In [7]:
!unzip glove6b100dtxt.zip -d glove100

Archive:  glove6b100dtxt.zip
  inflating: glove100/glove.6B.100d.txt  


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
max_seq_len = 100
max_vocab_size = 20000
epochs = 50
batch_size = 64
embedding_dim = 100
validation_split = 0.2
latent_dim = 25

In [ ]:
input_txts = []
target_txts = []

In [ ]:
for line in open('robert_frost.txt'):
  
  line = line.strip()
  input_line = '<SOS> ' + line
  target_line = line + ' <EOS>'

  input_txts.append(input_line)
  target_txts.append(target_line)

all_lines = input_txts + target_txts

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_vocab_size, filters='')
tokenizer.fit_on_texts(all_lines)
input_sequences = tokenizer.texts_to_sequences(input_txts)
target_sequences = tokenizer.texts_to_sequences(target_txts)

In [ ]:
max_seq_len_lines = max(len(s) for s in all_lines)
max_seq_len_lines

64

In [ ]:
word2index = tokenizer.word_index
print(len(word2index))

4620


In [ ]:
assert '<sos>' in word2index
assert '<eos>' in word2index

In [ ]:
max_seq_len = min(max_seq_len, max_seq_len_lines)
max_seq_len

64

In [ ]:
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_seq_len, padding='post')
target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, maxlen=max_seq_len, padding='post')

In [ ]:
input_sequences.shape

(3162, 64)

In [ ]:
target_sequences.shape

(3162, 64)

In [ ]:
word_embeddings = {}
with open(os.path.join('/content/glove100/glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word_key = values[0]
        word_vector = np.array(values[1:], dtype='float32')
        
        word_embeddings[word_key] = word_vector

In [ ]:
num_words = min(max_vocab_size, len(word2index)+1)
num_words

4621

In [ ]:
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, idx in word_index.items():
    if idx < max_vocab_size:
        embedding_vector = word_embeddings.get(word)
        
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector

In [ ]:
one_hot_targets = np.zeros((len(target_sequences), max_seq_len, num_words))
for i, target_sequence in enumerate(target_sequences):
  for j, token_id in enumerate(target_sequence):
      if token_id > 0:
        one_hot_targets[i, j, token_id] = 1

one_hot_targets.shape

(3162, 64, 4621)

In [ ]:
embedding_layer = tf.keras.layers.Embedding(input_dim=num_words, output_dim=embedding_dim, weights=embedding_matrix ,input_length=max_seq_len, trainable=False)